In [2]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers import *

Using TensorFlow backend.


In [3]:
input_tensor = Input((400, 80, 1))
x = input_tensor
for i in range(3):
    x = Conv2D(32 * 2 ** i, (3, 3), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(32 * 2 ** i, (3, 3), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

In [5]:
x.get_shape().as_list()

[None, 46, 6, 128]

In [6]:
irs = tf.InteractiveSession()

In [1]:
a = np.eye(10, 10)

NameError: name 'np' is not defined

In [9]:
tf.squeeze(a).eval()

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [2]:
import tensorflow as tf

In [5]:
tf.Tensor([None, 1, 2], 0, dtype=tf.float32)

TypeError: op needs to be an Operation: [None, 1, 2]

In [6]:
tf.convert_to_tensor([None, 1, 2])

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [None, 1, 2]. Consider casting elements to a supported type.

In [7]:
x = tf.placeholder(tf.float32, shape=[None, 1, 2], name='x')

In [9]:
tf.random_normal([None, 1, 2][1:2], stddev=0.01)

<tf.Tensor 'random_normal_1:0' shape=(1,) dtype=float32>

In [ ]:
tf.random_uniform([None, self.embedding_size], -1., 1.)